## Capítulo 02: Integração DuckDB + Arrow
**Curso:** Apache Arrow + DuckDB

### Tópicos abordados:
- Modos de integração
- Conversões bidirecionais
- Query em Arrow Tables
- Schemas e metadados
- Error handling

In [11]:
# -*- coding: utf-8 -*-
"""
Configuração inicial e imports
Nota: UTF-8 é configurado automaticamente em notebooks Jupyter
"""

import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np

# Instalar dependências
print("Dependências necessárias:")
print("pip install pyarrow duckdb pandas numpy")

print("="*60)
print(f"CAPÍTULO 02: INTEGRAÇÃO DUCKDB + ARROW")
print("="*60)

Dependências necessárias:
pip install pyarrow duckdb pandas numpy
CAPÍTULO 02: INTEGRAÇÃO DUCKDB + ARROW


### Preparação dos Dados

In [12]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()


Gerando dados de exemplo...
Tabela PyArrow criada: 1000 linhas


### Tópico 1: Modos de integração

In [13]:
print(f"\n--- {'Modos de integração'.upper()} ---")

# 2.1.1 Query Direto em Arrow Tables
print("\n1. Query Direto em Arrow Tables:")
print("-" * 40)

# Criar Arrow table
customers = pa.table({
    'customer_id': [1, 2, 3, 4, 5],
    'name': ['Alice', 'Bob', 'Carol', 'David', 'Eve'],
    'country': ['USA', 'UK', 'USA', 'Canada', 'UK'],
    'total_orders': [10, 5, 15, 8, 12]
})

# Método 1: Query direta (variável em escopo)
result = con.execute("""
    SELECT country, sum(total_orders) as total
    FROM customers
    GROUP BY country
    ORDER BY total DESC
""").arrow()

print("Resultado como Arrow:")
print(result)

# 2.1.2 Registrar Arrow Tables
print("\n2. Registrar Arrow Tables:")
print("-" * 40)

# Criar múltiplas Arrow tables
products = pa.table({
    'product_id': [101, 102, 103],
    'product_name': ['Laptop', 'Mouse', 'Keyboard'],
    'price': [999.99, 29.99, 79.99]
})

orders = pa.table({
    'order_id': [1, 2, 3, 4],
    'product_id': [101, 102, 101, 103],
    'quantity': [2, 5, 1, 3]
})

# Registrar explicitamente
con.register('products_table', products)
con.register('orders_table', orders)

# Fazer JOIN entre tables
result_join = con.execute("""
    SELECT
        p.product_name,
        o.quantity,
        p.price,
        o.quantity * p.price as total
    FROM orders_table o
    JOIN products_table p ON o.product_id = p.product_id
""").df()

print(result_join)

# 2.1.3 Arrow Views (Sem Cópia)
print("\n3. Arrow Views (Sem Cópia - Zero-Copy):")
print("-" * 40)

# Criar Arrow table grande
large_table = pa.table({
    'id': range(100_000),
    'value': [i * 1.5 for i in range(100_000)]
})

# DuckDB cria uma VIEW sobre os dados Arrow - Não há cópia!
con.execute("CREATE OR REPLACE VIEW arrow_view AS SELECT * FROM large_table")

# Query na view
result_view = con.execute("""
    SELECT
        count(*) as count,
        avg(value) as avg_value,
        min(value) as min_value,
        max(value) as max_value
    FROM arrow_view
    WHERE value > 100000
""").fetchone()

print(f"Count: {result_view[0]:,}")
print(f"Average: {result_view[1]:,.2f}")
print(f"Min: {result_view[2]:,.2f}")
print(f"Max: {result_view[3]:,.2f}")


--- MODOS DE INTEGRAÇÃO ---

1. Query Direto em Arrow Tables:
----------------------------------------
Resultado como Arrow:
pyarrow.Table
country: string
total: decimal128(38, 0)
----
country: [["USA","UK","Canada"]]
total: [[25,17,8]]

2. Registrar Arrow Tables:
----------------------------------------
  product_name  quantity   price    total
0       Laptop         1  999.99   999.99
1        Mouse         5   29.99   149.95
2     Keyboard         3   79.99   239.97
3       Laptop         2  999.99  1999.98

3. Arrow Views (Sem Cópia - Zero-Copy):
----------------------------------------
Count: 33,333
Average: 124,999.50
Min: 100,000.50
Max: 149,998.50


### Tópico 2: Conversões bidirecionais

In [14]:
print(f"\n--- {'Conversões bidirecionais'.upper()} ---")

# 2.2.1 .arrow() - Retornar como Arrow Table
print("\n1. Retornar como Arrow Table (.arrow()):")
print("-" * 40)

data_sales = pa.table({
    'product': ['A', 'B', 'C', 'D'],
    'sales': [100, 200, 150, 300]
})

# Retornar como Arrow table
result_arrow = con.execute("""
    SELECT product, sales
    FROM data_sales
    WHERE sales > 100
    ORDER BY sales DESC
""").arrow()

print(f"Tipo: {type(result_arrow)}")
print(result_arrow)

# Acessar colunas Arrow
print(f"\nProdutos: {result_arrow['product'].to_pylist()}")
print(f"Vendas: {result_arrow['sales'].to_pylist()}")

# 2.2.2 .df() - Retornar como Pandas
print("\n2. Retornar como Pandas DataFrame (.df()):")
print("-" * 40)

data_revenue = pa.table({
    'date': pa.array(['2024-01-01', '2024-01-02', '2024-01-03'], type=pa.string()),
    'revenue': [1000, 1500, 1200]
})

# Retornar como Pandas DataFrame
result_df = con.execute("""
    SELECT date, revenue
    FROM data_revenue
    ORDER BY revenue DESC
""").df()

print(f"Tipo: {type(result_df)}")
print(result_df)
print(f"\nSoma total: ${result_df['revenue'].sum():,}")

# 2.2.3 .fetchall() / .fetchone() - Retornar como Python
print("\n3. Retornar como Python nativo (.fetchall(), .fetchone()):")
print("-" * 40)

data_category = pa.table({
    'category': ['Electronics', 'Clothing', 'Food'],
    'count': [50, 30, 100]
})

# Retornar como lista de tuplas Python
all_rows = con.execute("SELECT * FROM data_category").fetchall()
print(f"Todas as linhas: {all_rows}")

# Retornar uma linha
one_row = con.execute("SELECT * FROM data_category WHERE count > 40").fetchone()
print(f"Uma linha: {one_row}")

# Retornar muitas linhas (com limite)
many_rows = con.execute("SELECT * FROM data_category").fetchmany(size=2)
print(f"Primeiras 2 linhas: {many_rows}")

# 2.3 Arrow Record Batches
print("\n4. Trabalhando com Record Batches:")
print("-" * 40)

# Criar record batches (streaming)
batch1 = pa.record_batch([
    pa.array([1, 2, 3]),
    pa.array(['A', 'B', 'C'])
], names=['id', 'category'])

batch2 = pa.record_batch([
    pa.array([4, 5, 6]),
    pa.array(['D', 'E', 'F'])
], names=['id', 'category'])

# Criar table de batches
batches = [batch1, batch2]
schema = pa.schema([
    ('id', pa.int64()),
    ('category', pa.string())
])
table_from_batches = pa.Table.from_batches(batches, schema=schema)

print("Table criada de batches:")
print(table_from_batches)

# Query com DuckDB
result_batches = con.execute("SELECT * FROM table_from_batches ORDER BY id").arrow()
print("\nResultado:")
print(result_batches)


--- CONVERSÕES BIDIRECIONAIS ---

1. Retornar como Arrow Table (.arrow()):
----------------------------------------
Tipo: <class 'pyarrow.lib.Table'>
pyarrow.Table
product: string
sales: int64
----
product: [["D","B","C"]]
sales: [[300,200,150]]

Produtos: ['D', 'B', 'C']
Vendas: [300, 200, 150]

2. Retornar como Pandas DataFrame (.df()):
----------------------------------------
Tipo: <class 'pandas.core.frame.DataFrame'>
         date  revenue
0  2024-01-02     1500
1  2024-01-03     1200
2  2024-01-01     1000

Soma total: $3,700

3. Retornar como Python nativo (.fetchall(), .fetchone()):
----------------------------------------
Todas as linhas: [('Electronics', 50), ('Clothing', 30), ('Food', 100)]
Uma linha: ('Electronics', 50)
Primeiras 2 linhas: [('Electronics', 50), ('Clothing', 30)]

4. Trabalhando com Record Batches:
----------------------------------------
Table criada de batches:
pyarrow.Table
id: int64
category: string
----
id: [[1,2,3],[4,5,6]]
category: [["A","B","C"],["

### Tópico 3: Query em Arrow Tables

In [15]:
print(f"\n--- {'Query em Arrow Tables'.upper()} ---")

# 2.4.1 Queries SQL Complexas em Arrow
print("\n1. Queries SQL Complexas em Arrow Tables:")
print("-" * 40)

# Criar dados de vendas
sales_data = pa.table({
    'sale_id': range(1, 21),
    'product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Laptop'] * 4,
    'region': ['North', 'South', 'East', 'West', 'North'] * 4,
    'amount': [1200, 25, 80, 350, 1150, 1300, 30, 75, 400, 1100,
               1250, 28, 85, 375, 1175, 1280, 32, 78, 390, 1125],
    'quantity': [1, 2, 3, 1, 1, 1, 3, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1]
})

# Query agregada complexa
result_complex = con.execute("""
    SELECT
        product,
        region,
        count(*) as num_sales,
        sum(amount) as total_amount,
        avg(amount) as avg_amount,
        sum(quantity) as total_qty
    FROM sales_data
    GROUP BY product, region
    HAVING sum(amount) > 100
    ORDER BY total_amount DESC
    LIMIT 5
""").arrow()

print("Top 5 vendas por produto/região:")
print(result_complex)

# 2.4.2 Window Functions em Arrow
print("\n2. Window Functions em Arrow Tables:")
print("-" * 40)

# Query com window functions
result_window = con.execute("""
    SELECT
        sale_id,
        product,
        amount,
        row_number() OVER (PARTITION BY product ORDER BY amount DESC) as rank_in_product,
        sum(amount) OVER (PARTITION BY product) as total_per_product,
        avg(amount) OVER (PARTITION BY product) as avg_per_product
    FROM sales_data
    ORDER BY product, rank_in_product
""").df()

print("Ranking e agregações por janela:")
print(result_window.head(10))

# 2.4.3 Subqueries e CTEs
print("\n3. Subqueries e CTEs (Common Table Expressions):")
print("-" * 40)

# Query com CTE
result_cte = con.execute("""
    WITH product_stats AS (
        SELECT
            product,
            sum(amount) as total_sales,
            avg(amount) as avg_sales,
            count(*) as num_transactions
        FROM sales_data
        GROUP BY product
    ),
    region_stats AS (
        SELECT
            region,
            sum(amount) as total_sales,
            count(*) as num_transactions
        FROM sales_data
        GROUP BY region
    )
    SELECT
        p.product,
        p.total_sales as product_total,
        p.avg_sales,
        (SELECT sum(total_sales) FROM region_stats) as grand_total,
        ROUND(p.total_sales * 100.0 / (SELECT sum(total_sales) FROM region_stats), 2) as pct_of_total
    FROM product_stats p
    ORDER BY product_total DESC
""").arrow()

print("Análise com CTEs:")
print(result_cte)

# 2.4.4 JOINs entre múltiplas Arrow Tables
print("\n4. JOINs entre múltiplas Arrow Tables:")
print("-" * 40)

# Criar tabelas relacionadas
products_info = pa.table({
    'product': ['Laptop', 'Mouse', 'Keyboard', 'Monitor'],
    'category': ['Electronics', 'Accessories', 'Accessories', 'Electronics'],
    'cost': [800, 15, 40, 200]
})

regions_info = pa.table({
    'region': ['North', 'South', 'East', 'West'],
    'manager': ['Alice', 'Bob', 'Carol', 'David'],
    'target': [50000, 40000, 45000, 48000]
})

# Registrar as tabelas
con.register('products_info', products_info)
con.register('regions_info', regions_info)

# Query com múltiplos JOINs
result_joins = con.execute("""
    SELECT
        s.region,
        r.manager,
        s.product,
        p.category,
        sum(s.amount) as revenue,
        sum(s.quantity * p.cost) as total_cost,
        sum(s.amount) - sum(s.quantity * p.cost) as profit,
        r.target,
        sum(s.amount) * 100.0 / r.target as pct_target
    FROM sales_data s
    JOIN products_info p ON s.product = p.product
    JOIN regions_info r ON s.region = r.region
    GROUP BY s.region, r.manager, s.product, p.category, r.target
    ORDER BY profit DESC
""").df()

print("Análise de lucro por região/produto:")
print(result_joins.head(10))


--- QUERY EM ARROW TABLES ---

1. Queries SQL Complexas em Arrow Tables:
----------------------------------------
Top 5 vendas por produto/região:
pyarrow.Table
product: string
region: string
num_sales: int64
total_amount: decimal128(38, 0)
avg_amount: double
total_qty: decimal128(38, 0)
----
product: [["Laptop","Monitor","Keyboard","Mouse"]]
region: [["North","West","East","South"]]
num_sales: [[8,4,4,4]]
total_amount: [[9580,1515,318,115]]
avg_amount: [[1197.5,378.75,79.5,28.75]]
total_qty: [[8,4,11,9]]

2. Window Functions em Arrow Tables:
----------------------------------------
Ranking e agregações por janela:
   sale_id   product  amount  rank_in_product  total_per_product  \
0       13  Keyboard      85                1              318.0   
1        3  Keyboard      80                2              318.0   
2       18  Keyboard      78                3              318.0   
3        8  Keyboard      75                4              318.0   
4        6    Laptop    1300        

### Tópico 4: Schemas e metadados

In [16]:
print(f"\n--- {'Schemas e metadados'.upper()} ---")

# 2.5.1 Definir e Validar Schemas
print("\n1. Definir e Validar Schemas:")
print("-" * 40)

from datetime import date
from decimal import Decimal

# Definir schema explícito
schema = pa.schema([
    pa.field('employee_id', pa.int32(), nullable=False),
    pa.field('name', pa.string(), nullable=False),
    pa.field('department', pa.string(), nullable=True),
    pa.field('salary', pa.decimal128(10, 2), nullable=False),
    pa.field('hire_date', pa.date32(), nullable=False),
    pa.field('is_manager', pa.bool_(), nullable=False)
])

print("Schema definido:")
print(schema)

# Criar table com schema
data_arrays = [
    pa.array([1, 2, 3], type=pa.int32()),
    pa.array(['Alice', 'Bob', 'Carol'], type=pa.string()),
    pa.array(['HR', 'IT', 'Sales'], type=pa.string()),
    pa.array([Decimal('75000.00'), Decimal('85000.00'), Decimal('70000.00')], type=pa.decimal128(10, 2)),
    pa.array([18628, 18700, 18750], type=pa.date32()),  # Dias desde epoch
    pa.array([True, False, True], type=pa.bool_())
]

employees = pa.Table.from_arrays(data_arrays, schema=schema)
print("\nTabela criada:")
print(employees)

# Query com DuckDB
result_schema = con.execute("""
    SELECT
        name,
        department,
        salary,
        is_manager
    FROM employees
    WHERE salary > 70000
    ORDER BY salary DESC
""").arrow()

print("\nResultado:")
print(result_schema)

# 2.5.2 Modificar Schemas
print("\n2. Modificar Schemas:")
print("-" * 40)

# Table original
original = pa.table({
    'id': [1, 2, 3],
    'name': ['A', 'B', 'C'],
    'value': [10, 20, 30]
})

print("Schema original:")
print(original.schema)

# Renomear colunas
renamed = original.rename_columns(['product_id', 'product_name', 'quantity'])
print("\nSchema renomeado:")
print(renamed.schema)

# Selecionar colunas específicas
selected = original.select(['id', 'name'])
print("\nSchema com colunas selecionadas:")
print(selected.schema)

# Adicionar coluna computada com DuckDB
con.register('original', original)
with_computed = con.execute("""
    SELECT
        *,
        value * 2 as doubled_value,
        value > 15 as is_high
    FROM original
""").arrow()

print("\nSchema com colunas computadas:")
print(with_computed.schema)

# 2.5.3 Inspecionar Metadados
print("\n3. Inspecionar Metadados:")
print("-" * 40)

# Criar table com metadados customizados
custom_metadata = {
    b'author': b'Data Team',
    b'version': b'1.0',
    b'description': b'Sales data 2024'
}

schema_with_metadata = pa.schema([
    pa.field('id', pa.int32()),
    pa.field('sales', pa.float64())
], metadata=custom_metadata)

table_with_metadata = pa.table({
    'id': [1, 2, 3],
    'sales': [100.5, 200.3, 150.8]
}, schema=schema_with_metadata)

print("Schema com metadados:")
print(table_with_metadata.schema)
print("\nMetadados customizados:")
for key, value in table_with_metadata.schema.metadata.items():
    print(f"  {key.decode()}: {value.decode()}")

# 2.5.4 Verificar Tipos de Dados
print("\n4. Verificar Tipos de Dados:")
print("-" * 40)

# Criar table com tipos variados
mixed_types = pa.table({
    'int_col': pa.array([1, 2, 3], type=pa.int64()),
    'float_col': pa.array([1.1, 2.2, 3.3], type=pa.float64()),
    'string_col': pa.array(['a', 'b', 'c'], type=pa.string()),
    'bool_col': pa.array([True, False, True], type=pa.bool_()),
    'date_col': pa.array([date(2024, 1, 1), date(2024, 1, 2), date(2024, 1, 3)], type=pa.date32())
})

print("Tipos de dados de cada coluna:")
for i, field in enumerate(mixed_types.schema):
    print(f"  {field.name}: {field.type} (nullable={field.nullable})")

# Verificar compatibilidade com DuckDB
result_types = con.execute("SELECT * FROM mixed_types").arrow()
print("\nSchema após query no DuckDB:")
for field in result_types.schema:
    print(f"  {field.name}: {field.type}")

# 2.5.5 Schema Evolution
print("\n5. Schema Evolution (Evolução de Schema):")
print("-" * 40)

# Schema v1
schema_v1 = pa.schema([
    pa.field('id', pa.int32()),
    pa.field('name', pa.string())
])

table_v1 = pa.table({
    'id': [1, 2],
    'name': ['Alice', 'Bob']
}, schema=schema_v1)

print("Schema v1:")
print(table_v1.schema)

# Schema v2 - Adicionar novas colunas com DuckDB
result_v2 = con.execute("""
    SELECT
        *,
        0.0 as balance,
        true as active,
        CURRENT_DATE as created_date
    FROM table_v1
""").arrow()

print("\nSchema v2 (com novas colunas):")
print(result_v2.schema)
print("\nDados:")
print(result_v2)


--- SCHEMAS E METADADOS ---

1. Definir e Validar Schemas:
----------------------------------------
Schema definido:
employee_id: int32 not null
name: string not null
department: string
salary: decimal128(10, 2) not null
hire_date: date32[day] not null
is_manager: bool not null

Tabela criada:
pyarrow.Table
employee_id: int32 not null
name: string not null
department: string
salary: decimal128(10, 2) not null
hire_date: date32[day] not null
is_manager: bool not null
----
employee_id: [[1,2,3]]
name: [["Alice","Bob","Carol"]]
department: [["HR","IT","Sales"]]
salary: [[75000.00,85000.00,70000.00]]
hire_date: [[2021-01-01,2021-03-14,2021-05-03]]
is_manager: [[true,false,true]]

Resultado:
pyarrow.Table
name: string
department: string
salary: decimal128(10, 2)
is_manager: bool
----
name: [["Bob","Alice"]]
department: [["IT","HR"]]
salary: [[85000.00,75000.00]]
is_manager: [[false,true]]

2. Modificar Schemas:
----------------------------------------
Schema original:
id: int64
name: strin

### Tópico 5: Error handling

In [ ]:
print(f"\n--- {'Error handling'.upper()} ---")

# 2.7.1 Tratamento de Erros Arrow
print("\n1. Tratamento de Erros em Queries Arrow:")
print("-" * 40)

def safe_arrow_query(table, query):
    """Executa query com tratamento de erro"""
    try:
        # Registrar table
        con.register('query_table', table)
        
        # Executar query
        result = con.execute(query).arrow()
        return result, None
    
    except Exception as e:
        return None, str(e)

# Teste 1: Query válida
data_test = pa.table({'x': [1, 2, 3], 'y': [4, 5, 6]})
result, error = safe_arrow_query(data_test, "SELECT x, y FROM query_table WHERE x > 1")

if error:
    print(f"Erro: {error}")
else:
    print("✓ Sucesso - Query válida:")
    print(result)

# Teste 2: Query inválida (coluna não existe)
result, error = safe_arrow_query(data_test, "SELECT z FROM query_table")

if error:
    print(f"\n✗ Erro esperado capturado: {error}")

# Teste 3: Erro de sintaxe SQL
result, error = safe_arrow_query(data_test, "SELEKT x FROM query_table")

if error:
    print(f"\n✗ Erro de sintaxe capturado: {error}")

# 2.7.2 Validação de Schema
print("\n2. Validação de Schema:")
print("-" * 40)

def validate_and_query(table, expected_columns):
    """Valida schema antes de query"""
    # Validar colunas
    actual_columns = set(table.schema.names)
    expected_set = set(expected_columns)
    
    if not expected_set.issubset(actual_columns):
        missing = expected_set - actual_columns
        raise ValueError(f"Colunas faltando: {missing}")
    
    # Se válido, fazer query
    result = con.execute(f"""
        SELECT {', '.join(expected_columns)}
        FROM table
    """).arrow()
    
    return result

# Teste com schema válido
data_valid = pa.table({
    'id': [1, 2, 3],
    'name': ['A', 'B', 'C'],
    'value': [10, 20, 30]
})

try:
    result = validate_and_query(data_valid, ['id', 'name'])
    print("✓ Validação passou:")
    print(result)
except ValueError as e:
    print(f"Erro de validação: {e}")

# Teste com coluna faltando
try:
    result = validate_and_query(data_valid, ['id', 'missing_column'])
    print("Validação passou:")
    print(result)
except ValueError as e:
    print(f"\n✗ Erro de validação esperado: {e}")

# 2.7.3 Validação de Tipos de Dados
print("\n3. Validação de Tipos de Dados:")
print("-" * 40)

def validate_types(table, expected_types):
    """Valida tipos de dados do schema"""
    errors = []
    
    for col_name, expected_type in expected_types.items():
        if col_name not in table.schema.names:
            errors.append(f"Coluna '{col_name}' não encontrada")
            continue
        
        field = table.schema.field(col_name)
        if str(field.type) != expected_type:
            errors.append(
                f"Tipo incorreto para '{col_name}': "
                f"esperado {expected_type}, obtido {field.type}"
            )
    
    return errors

# Criar table para teste
test_table = pa.table({
    'id': pa.array([1, 2, 3], type=pa.int32()),
    'amount': pa.array([10.5, 20.3, 30.7], type=pa.float64()),
    'active': pa.array([True, False, True], type=pa.bool_())
})

# Validar tipos corretos
expected_correct = {
    'id': 'int32',
    'amount': 'double',
    'active': 'bool'
}

errors = validate_types(test_table, expected_correct)
if errors:
    print("✗ Erros encontrados:")
    for error in errors:
        print(f"  - {error}")
else:
    print("✓ Todos os tipos estão corretos!")

# Validar tipos incorretos
expected_wrong = {
    'id': 'int64',  # Tipo errado
    'amount': 'double',
    'missing': 'string'  # Coluna inexistente
}

errors = validate_types(test_table, expected_wrong)
if errors:
    print("\n✗ Erros esperados encontrados:")
    for error in errors:
        print(f"  - {error}")

# 2.7.4 Try-Except com Conversões
print("\n4. Tratamento de Erros em Conversões:")
print("-" * 40)

def safe_conversion(data, target_format='arrow'):
    """Converte dados com tratamento de erro"""
    try:
        if target_format == 'arrow':
            # Pandas para Arrow
            if isinstance(data, pd.DataFrame):
                result = pa.Table.from_pandas(data)
                print(f"✓ Conversão Pandas → Arrow bem-sucedida")
                return result
            else:
                raise ValueError("Dados não são DataFrame Pandas")
        
        elif target_format == 'pandas':
            # Arrow para Pandas
            if isinstance(data, pa.Table):
                result = data.to_pandas()
                print(f"✓ Conversão Arrow → Pandas bem-sucedida")
                return result
            else:
                raise ValueError("Dados não são Arrow Table")
    
    except Exception as e:
        print(f"✗ Erro na conversão: {e}")
        return None

# Teste conversões
df_test = pd.DataFrame({'a': [1, 2, 3], 'b': [4, 5, 6]})
arrow_result = safe_conversion(df_test, 'arrow')

if arrow_result:
    pandas_result = safe_conversion(arrow_result, 'pandas')

# Teste com tipo errado
invalid_data = "string inválida"
safe_conversion(invalid_data, 'arrow')

# 2.7.5 Validação Completa de Pipeline
print("\n5. Validação Completa de Pipeline:")
print("-" * 40)

def safe_pipeline(data, operations):
    """Executa pipeline com validação em cada etapa"""
    print("Iniciando pipeline...")
    current_data = data
    
    for i, operation in enumerate(operations, 1):
        try:
            print(f"\n  Etapa {i}: {operation['name']}")
            
            # Validar entrada
            if not isinstance(current_data, pa.Table):
                raise TypeError("Dados devem ser Arrow Table")
            
            # Executar operação
            current_data = con.execute(operation['query']).arrow()
            print(f"  ✓ {current_data.num_rows} linhas processadas")
            
        except Exception as e:
            print(f"  ✗ Erro na etapa {i}: {e}")
            return None
    
    print("\n✓ Pipeline concluído com sucesso!")
    return current_data

# Criar dados de teste
pipeline_data = pa.table({
    'product': ['A', 'B', 'C', 'D', 'E'],
    'price': [100, 50, 200, 75, 150],
    'quantity': [10, 20, 5, 15, 8]
})

# Definir operações do pipeline
operations = [
    {
        'name': 'Filtrar preço > 60',
        'query': 'SELECT * FROM pipeline_data WHERE price > 60'
    },
    {
        'name': 'Calcular total',
        'query': 'SELECT product, price, quantity, price * quantity as total FROM pipeline_data'
    },
    {
        'name': 'Ordenar por total',
        'query': 'SELECT * FROM pipeline_data ORDER BY total DESC'
    }
]

# Executar pipeline
final_result = safe_pipeline(pipeline_data, operations)

if final_result:
    print("\nResultado final:")
    print(final_result)